In [ ]:
from src.models.utils import read_csv_data, clean_location

data = read_csv_data("../data/processed_job_postings.csv", ["name", "industry", "location", "formatted_experience_level"], "standardized_annual_salary")
data = clean_location(data, 2)


import random
random.seed(42)
random.shuffle(data)

train_data = data[:10000]
val_data = data[10000:11000]
test_data = data[12000:]

In [ ]:
print(data[10])

In [ ]:
from src.models.utils import build_column_vocabulary

vocab_name = build_column_vocabulary(train_data, 0)
vocab_sector = build_column_vocabulary(train_data, 1)
vocab_state = build_column_vocabulary(train_data, 2)
vocab_level = build_column_vocabulary(train_data, 3)
len(vocab_state)